<a href="https://colab.research.google.com/github/davidcode101/fantastic-spoon/blob/main/Input_target_pairs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tiktoken
# tokenizer=tiktoken.get_encoding("cl100k_base")
tokenizer=tiktoken.get_encoding("o200k_base")
# tokenizer=tiktoken.get_encoding("gpt2") for comparing
print(len(tokenizer._mergeable_ranks))

199998


In [3]:
with open ("/content/drive/MyDrive/Data for llm/TXT data/Kimi no Na wa [Yen Press].txt","r",encoding="utf-8") as f:
    raw_text = f.read()
enc_text= tokenizer.encode(raw_text)
print(len(enc_text))


60516


In [4]:
enc_sample=enc_text[50:]

In [5]:
context_size=4
x=enc_sample[:context_size]
y=enc_sample[1:context_size+1]
print(f"x:{x}")
print(f"y:{y}")

x:[6787, 198, 90172, 70453]
y:[198, 90172, 70453, 13]


In [6]:
for i in range(1,context_size+1):
  context=enc_sample[:i]
  desried=enc_sample[i]

  print(tokenizer.decode(context),"---->",tokenizer.decode([desried]))

 almost ----> 

 almost
 ----> inse
 almost
inse ----> parable
 almost
inseparable ----> .


In [7]:
import torch
from torch.utils.data import Dataset,DataLoader
class GptDatasetV1(Dataset):
  def __init__(self,txt,tokenizer,max_length,stride):
    self.input_ids=[]
    self.target_ids=[]
    #tokenizer
    token_ids=tokenizer.encode(txt,allowed_special={"<|endoftext|>"})
     #sliding window
    for i in range(0,len(token_ids)-max_length,stride):
      input_chunk=token_ids[i:i+max_length]
      target_chunk=token_ids[i+1:i+max_length+1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))
  def __len__(self):
    return len(self.input_ids)
  def __getitem__(self,idx):
    return self.input_ids[idx],self.target_ids[idx]

In [8]:
def create_dataloader_v1(txt,batch_size=4,max_length=256,stride=128,
                         shuffle=True,drop_last=True,num_workers=0):
     tokenizer=tiktoken.get_encoding("o200k_base")
     dataset=GptDatasetV1(txt,tokenizer,max_length,stride)
     dataloader=DataLoader(dataset,batch_size=batch_size,shuffle=shuffle,drop_last=drop_last,num_workers=num_workers)
     return dataloader

In [9]:
with open ("/content/drive/MyDrive/Data for llm/TXT data/Kimi no Na wa [Yen Press].txt","r",encoding="utf-8") as f:
    raw_text = f.read()

In [10]:
import torch
print("Torch version",torch.__version__)
dataloader=create_dataloader_v1(
    raw_text,batch_size=1,max_length=4,stride=1,
    shuffle=False
)
data_iter=iter(dataloader)
first_batch=next(data_iter)
print(first_batch)

Torch version 2.8.0+cu126
[tensor([[200, 200, 198, 200]]), tensor([[200, 198, 200, 200]])]


In [11]:
second_batch=next(data_iter)
print(second_batch)

[tensor([[200, 198, 200, 200]]), tensor([[  198,   200,   200, 45990]])]


In [12]:
dataloader=create_dataloader_v1(
    raw_text,batch_size=8,max_length=256,stride=4,
    shuffle=False
)
data_iter=iter(dataloader)
inputs,targets=next(data_iter)
print("inputs:'\n",inputs)
print("targets:'\n",targets)

inputs:'
 tensor([[  200,   200,   198,  ..., 17659,   290,  9807],
        [  200, 45990,  5108,  ...,   198,  3086, 37095],
        [77558,   198,    32,  ..., 51088, 14255,   922],
        ...,
        [50564,   558,    40,  ...,  7504,  3543,  2715],
        [34335,  4372,  5693,  ...,  4369,   200,    40],
        [ 3582,   316,   668,  ...,  1573,  1761,   558]])
targets:'
 tensor([[   200,    198,    200,  ...,    290,   9807,    326],
        [ 45990,   5108,    279,  ...,   3086,  37095,    484],
        [   198,     32, 132847,  ...,  14255,    922,   9623],
        ...,
        [   558,     40,   4572,  ...,   3543,   2715,  31946],
        [  4372,   5693,   1869,  ...,    200,     40,   1700],
        [   316,    668,     11,  ...,   1761,    558,     40]])


In [15]:
input_ids= torch.tensor([2,3,5,1])

In [17]:
vocab_size=6
output_dim=3
torch.manual_seed(123)
embedding_layer=torch.nn.Embedding(vocab_size,output_dim)


In [21]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [20]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [22]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)
